In [ ]:
import pandas as pd
import re
# Carga de archivos con pandas
# Para fines ilustrativos el archivo csv tiene diferentes formatos en las diferentes columnas, 
#esto para hacer mas grafico el proceso de tranformacion, 
df = pd.read_csv('ventas_brutas.csv', sep=';', decimal=',') 
print("Datos cargados. Filas originales:", len(df))

# Proceso de ETL
# Paso 1: Limpieza de la columna 'Price' y conversión a flotante,(Ej. "$4")
df['Price'] = df['Price'].str.replace('$', '').astype(float)

# Paso 2: Manejo de valores nulos en 'Quantity' (ej. rellenar con la mediana)
median_quantity = df['Quantity'].median()
df['Quantity'].fillna(median_quantity, inplace=True)

# Paso 3: Estandarización de la columna 'ProductCategory' (
df['ProductCategory'] = df['ProductCategory'].str.lower().str.strip()
df['ProductCategory'] = df['ProductCategory'].replace('electrónica', 'electronica')
df['ProductCategory'] = df['ProductCategory'].replace('áliménto', 'alimento')
df['ProductCategory'] = df['ProductCategory'].replace('vestímentá', 'vestimenta')

# Paso 4: Creación de una nueva característica
# Convertir 'Date' a datetime y calcular el 'TotalSale'
# Usa una columna auxiliar para preservar el original, se agrega al final
df['Date_raw'] = df['Date']
df['Date'] = df['Date_raw'].apply(try_parse_date)

# Reporte para depurar

df['Date'] = pd.to_datetime(df['Date'].astype(str).str.strip(), dayfirst=True, errors='coerce')
df['TotalSale'] = df['Price'] * df['Quantity']

# Paso 5: Agregación
# agrupar las ventas totales por dias y categoria para exportarlas
df_transformado = df.groupby(['Date', 'ProductCategory'])['TotalSale'].sum().reset_index()
df_transformado.rename(columns={'TotalSale': 'DailyCategoryTotal'}, inplace=True)
print("Datos transformados. Filas agregadas:", len(df_transformado))

# Cargar los datos transformados a un nuevo archivo CSV para su manejo en otros entornos, ejm Excel o Power Bi
df_transformado.to_csv('ventas_por_dia.csv', index=False, sep=';')
df.to_csv('ventas_limpias.csv', index=False, sep=';')

print("Datos cargados exitosamente al archivo 'ventas_por_dia.csv.csv'")

Datos cargados. Filas originales: 2809


C:\Users\User\AppData\Local\Temp\ipykernel_12424\1309108929.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantity'].fillna(median_quantity, inplace=True)


Datos transformados. Filas agregadas: 896
Datos cargados exitosamente al archivo 'ventas_por_dia.csv.csv'


C:\Users\User\AppData\Local\Temp\ipykernel_12424\1309108929.py:31: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'].astype(str).str.strip(), dayfirst=True, errors='coerce')
